In [ ]:
# Ignorar avisos
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importando bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, precision_recall_curve, roc_curve, auc

In [ ]:
# Carregando os dados
# Baixe o data frame em: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
caminho_dados = r'C:\Users\Matheus Fabiao\Desktop\TCC-IA-MERCADO-FINANCEIRO\data\creditcard.csv'
base_dados = pd.read_csv(caminho_dados)

In [ ]:
# Faz uma cópia dos dados para manter a base original intacta
dados = base_dados.copy()

In [ ]:
# Explorando os dados
dados.head()

In [ ]:
# Exibindo informações sobre o DataFrame
dados.info()

In [ ]:
# Resumindo estatísticas descritivas do DataFrame
dados.describe()

In [ ]:
# Verificando se há valores nulos nos dados
dados.isnull().sum().any()

In [ ]:
# Removendo a coluna 'Time', por não ser relevante
dados = dados.drop(['Time'], axis=1)

In [ ]:
# Separando as transações em fraude e não fraude
nao_fraudes = dados[dados['Class']==0]
print(f'O número de transações válidas é de {len(nao_fraudes)}')

In [ ]:
# Separando as transações em fraude e não fraude
fraudes = dados[dados['Class']==1]
print(f'O número de transações fraudulentas é de {len(fraudes)}')

In [ ]:
# Criando um conjunto de validação com 15 transações de cada classe
validacao_nao_fraudes = nao_fraudes.sample(15)
validacao_fraudes = fraudes.sample(15)

In [ ]:
# Concatenando os dois conjuntos de validação
validacao = pd.concat([validacao_nao_fraudes, validacao_fraudes], ignore_index=True)

In [ ]:
# Removendo as transações de validação do DataFrame original
dados = dados.drop(validacao_nao_fraudes.index)
dados = dados.drop(validacao_fraudes.index)

In [ ]:
# Criando o DataFrame com os valores reais, para comparar com as previsões
validacao_real = validacao.Class

In [ ]:
# Remover a coluna alvo 'Class'
validacao = validacao.drop(['Class'], axis=1)

In [ ]:
# Verificação da contagem das classes
# Desequilíbrio entre a minoritária e majoritária
dados['Class'].value_counts()

In [ ]:
# Separando os dados
x = dados.drop(['Class'], axis=1)
y = dados['Class']

In [ ]:
# Importação da biblioteca de oversampling
from imblearn.over_sampling import SMOTE

# Utilização do SMOTE para a sobreamostragem
smote = SMOTE(random_state=42)
X_reamostrado, y_reamostrado = smote.fit_resample(x, y)

In [ ]:
# Exibindo a contagem de rótulos após o SMOTE
# Equilíbrio entre a minoritária e majoritária
y_reamostrado.value_counts()

In [ ]:
# Dividindo os dados em conjuntos de treinamento e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(X_reamostrado, y_reamostrado, test_size=0.2, random_state=42)

In [ ]:
# Treinando o modelo de Regressão Logística
modelo = LogisticRegression(max_iter=1000)
modelo.fit(x_treino, y_treino)

In [ ]:
# Realizando previsões com o modelo
previsao = modelo.predict(x_teste)

In [ ]:
# Avaliando o modelo usando as principais métricas
precisao = precision_score(y_teste, previsao)
recall = recall_score(y_teste, previsao)
f1 = f1_score(y_teste, previsao)
acuracia = accuracy_score(y_teste, previsao)

In [ ]:
# Exibindo as principais métricas
print(f'Precisão: {precisao:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'A acurácia do modelo foi de {acuracia:.2f}')

In [ ]:
# Matriz de Confusão (avaliação visual do desempenho do modelo)
confusion = confusion_matrix(y_teste, previsao)
print("Matriz de Confusão:")
print(confusion)

In [ ]:
# Classification Report (resumo das métricas de classificação)
print("Classification Report:")
print(classification_report(y_teste, previsao))

In [ ]:
# # Validação cruzada
# pontuacoes_validacao_cruzada = cross_val_score(estimator=modelo, X=x_treino, y=y_treino, cv=5)
# print("Média da Validação Cruzada:", pontuacoes_validacao_cruzada.mean())

In [ ]:
# Precision-Recall Curve (trade-off entre precisão e recall)
precision, recall, thresholds = precision_recall_curve(y_teste, modelo.predict_proba(x_teste)[:, 1])
plt.style.use('fivethirtyeight')
plt.figure(figsize=(16, 8))
plt.plot(recall, precision, color='darkorange', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

In [ ]:
# Calculando a Curva ROC
fpr, tpr, thresholds = roc_curve(y_teste, previsao)
# Calculando a Área sob a Curva ROC (AUC)
roc_auc = auc(fpr, tpr)
# Plotando a Curva ROC
plt.figure(figsize=(16, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Fazendo previsões no conjunto de validação
predict = modelo.predict(validacao)

# Criando um DataFrame para comparar as classes reais com as previsões
dados = pd.DataFrame({'real':validacao_real, 'previsao':predict})

In [ ]:
# Exibindo o DataFrame de comparação
dados